In [ ]:
!pip install torch torchvision kagglehub pandas scikit-learn tqdm

In [ ]:
import torch
import torch.nn as nn
import numpy as np

from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
USE_AMP = torch.cuda.is_available()
print("Using device:", device)

Using device: cuda


In [ ]:
import kagglehub

dataset = kagglehub.dataset_download(
    "xhlulu/140k-real-and-fake-faces"
)

print(dataset)

100%|██████████| 3.75G/3.75G [02:54<00:00, 23.1MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2


In [ ]:
import pandas as pd

DATASET_ROOT = dataset

train_df = pd.read_csv(f"{DATASET_ROOT}/train.csv")
valid_df = pd.read_csv(f"{DATASET_ROOT}/valid.csv")

print(train_df.head())


   Unnamed: 0                                      original_path     id  \
0           0  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  31355   
1           1  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  02884   
2           2  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  33988   
3           3  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  53875   
4           4  /kaggle/input/flickrfaceshq-dataset-nvidia-par...  24149   

   label label_str                  path  
0      1      real  train/real/31355.jpg  
1      1      real  train/real/02884.jpg  
2      1      real  train/real/33988.jpg  
3      1      real  train/real/53875.jpg  
4      1      real  train/real/24149.jpg  


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class FaceDataset(Dataset):

    def __init__(self, dataframe, root_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        row = self.df.iloc[idx]

        img_rel_path = row["path"]
        label = int(row["label"])

        img_path = os.path.join(
            self.root_dir,
            "real_vs_fake",
            "real-vs-fake",
            img_rel_path
        )

        if not os.path.exists(img_path):
            raise FileNotFoundError(img_path)

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )
])

In [ ]:
train_dataset = FaceDataset(train_df, DATASET_ROOT, train_transform)
valid_dataset = FaceDataset(valid_df, DATASET_ROOT, val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)


val_loader = DataLoader(
    valid_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)



In [ ]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.amp import autocast, GradScaler

scaler = GradScaler("cuda", enabled=USE_AMP)

weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(
    weights=EfficientNet_B0_Weights.IMAGENET1K_V1
)

model.classifier[1] = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model.classifier[1].in_features, 1)
)
for param in model.features.parameters():
    param.requires_grad = False

model = model.to(device)

# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0]).to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 181MB/s]


In [ ]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, labels in tqdm(loader, desc="Training"):

        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad(set_to_none=True)

        if USE_AMP:
            with autocast("cuda"):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        else:
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [ ]:
def evaluate(model, loader):
    model.eval()
    preds, targets = [], []

    with torch.inference_mode():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            probs = torch.sigmoid(outputs)

            preds.extend(probs.cpu().numpy().ravel())
            targets.extend(labels.cpu().numpy().ravel())

    preds = np.array(preds)
    targets = np.array(targets)

    binary_preds = (preds > 0.4).astype(int)

    metrics = {
        "acc": accuracy_score(targets, binary_preds),
        "precision": precision_score(targets, binary_preds, zero_division=0),
        "recall": recall_score(targets, binary_preds, zero_division=0),
        "f1": f1_score(targets, binary_preds, zero_division=0),
        "auc": roc_auc_score(targets, preds)
    }

    return metrics


In [ ]:
trainable = [p for p in model.parameters() if p.requires_grad]
print("Trainable params:", len(trainable))


In [ ]:
EPOCHS = 7

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train_loss = train_one_epoch(model, train_loader)
    metrics = evaluate(model, val_loader)

    print(f"Train Loss: {train_loss:.4f}")
    print(
        f"Val Acc: {metrics['acc']:.4f}, "
        f"Precision: {metrics['precision']:.4f}, "
        f"Recall: {metrics['recall']:.4f}, "
        f"F1: {metrics['f1']:.4f}, "
        f"AUC: {metrics['auc']:.4f}"
)



Epoch 1/7


Training: 100%|██████████| 3125/3125 [12:31<00:00,  4.16it/s]


Train Loss: 0.5820
Val Acc: 0.7538, Precision: 0.8115, Recall: 0.6612, F1: 0.7287, AUC: 0.8478

Epoch 2/7


Training: 100%|██████████| 3125/3125 [12:21<00:00,  4.21it/s]


Train Loss: 0.5297
Val Acc: 0.7729, Precision: 0.8211, Recall: 0.6978, F1: 0.7545, AUC: 0.8641

Epoch 3/7


...

Train Loss: 0.5120
Val Acc: 0.7909, Precision: 0.8271, Recall: 0.7356, F1: 0.7787, AUC: 0.8804

Epoch 6/7


Training: 100%|██████████| 3125/3125 [11:54<00:00,  4.37it/s]


Train Loss: 0.5087
Val Acc: 0.7862, Precision: 0.8432, Recall: 0.7032, F1: 0.7668, AUC: 0.8818

Epoch 7/7


Training: 100%|██████████| 3125/3125 [11:57<00:00,  4.36it/s]


Train Loss: 0.5088
Val Acc: 0.7944, Precision: 0.8323, Recall: 0.7373, F1: 0.7819, AUC: 0.8831


In [ ]:
torch.save(model.state_dict(), "vigilant_eye.pth")